In [1]:
import os
import pandas as pd
import random
from copy import deepcopy
os.chdir(os.pardir) # comment if you are running this more than once
data_matrix_df = pd.read_csv('data/distance_matrix_official.csv', index_col=0)
data_matrix_np = data_matrix_df.to_numpy()
data_matrix_np.shape

(68, 68)

In [2]:
from Genetic_algorithm.fitness import ResourceFitness
from Genetic_algorithm.genome import Genome
from Genetic_algorithm.solution_rd import SolutionRD
from Genetic_algorithm.mutations import logistic_mutation, social_mutation, logistic_mutation_2
from Genetic_algorithm.crossovers import social_crossover, logistic_crossover_2, full_crossover
from Genetic_algorithm.selection_alogrithms import tournament_selection,rank_selection

my_fitness = ResourceFitness(data_matrix_np)

first_solution = SolutionRD(my_fitness, Genome, [logistic_mutation, social_mutation], [logistic_crossover_2, social_crossover])
secomnd_solution = SolutionRD(my_fitness, Genome, [logistic_mutation, social_mutation], [logistic_crossover_2, social_crossover])

In [ ]:
import tqdm
def genetic_algorithm(
    gen_count: int,
    selection_algorithm: list[callable],
    mutation_algorithms: list[callable],
    crossover_algorithms: list[callable],
    fitness_instance: ResourceFitness,
    Genome_class: Genome,
    maximization: bool = False,
    xo_prob: float = 0.9,
    mut_prob: float = 0.2,
    social_mutation_prob: float = 0.2,
    social_crossover_prob: float = 0.2,
    population_size: int = 10,
    elitism: bool = True,
    save_logs: bool = False,
):
    """
    Genetic Algorithm for solving optimization problems.
    This function implements a genetic algorithm that evolves a population of solutions
    over a specified number of generations. It uses selection, mutation, and crossover
    algorithms to create new solutions and improve the population's fitness.
    The algorithm can be configured with various parameters, including the number of
    generations, selection method, mutation and crossover probabilities, and whether
    to use elitism.

    Args:
        gen_count (int): number of generations to evolve.
        selection_algorithm (list[callable]): selection algorithm to use.
        mutation_algorithms (list[callable]): choose from Genetic_algotithm.mutations
        crossover_algorithms (list[callable]): choose from Genetic_algorithm.crossovers
        fitness_instance (ResourceFitness): instancee of the fitness class (it's important to instanciate it before as it holds the data matrix)
        Genome_class (Genome): class of the genome to be used (will be instanciated in the function)
        maximization (bool, optional): _description_. Defaults to False.
        xo_prob (float, optional): _description_. Defaults to 0.9.
        mut_prob (float, optional): _description_. Defaults to 0.2.
        social_mutation_prob (float, optional): _description_. Defaults to 0.2.
        social_crossover_prob (float, optional): _description_. Defaults to 0.2.
        elitism (bool, optional): _description_. Defaults to True.
        verbose (bool, optional): _description_. Defaults to False.
        
    """
    
    
    
    first_solution = SolutionRD(fitness_instance, Genome_class, mutation_algorithms, crossover_algorithms)
    
    first_solution.prob_social_mutation = social_mutation_prob
    first_solution.prob_social_crossover = social_crossover_prob
    first_solution.initial_population = population_size
    
    GA_population = [random_solution for random_solution in first_solution]
    
        
    for generation in tqdm.tqdm(range(gen_count), desc="Runnig genetic alogrithm", unit=" generation"):
        fitness_instance.number_of_calls = 0
        if save_logs:
            logs = []
        # Selection
        selected_individuals = [selection_algorithm(GA_population, maximization) for _ in range(population_size)]
        

        # Crossover
        new_population = []
        for i in range(0, population_size):
            for j in range(i+1, population_size):
                if random.random() < xo_prob:
                    parent1 = selected_individuals[i]
                    parent2 = selected_individuals[j]
                    child1, child2 = parent1 @ parent2
                    new_population.append(child1)
                    new_population.append(child2)
        
        if not new_population:
            new_population = selected_individuals.copy()
                   
        
        # Mutation
        for individual in new_population:
            if random.random() < mut_prob:
                individual.mutation()
        
        # Elitism
        if elitism:
            best_individual = min(new_population) if not maximization else max(new_population)
            new_population[0] = best_individual
        
        
        GA_population = new_population
        if save_logs:
            
            n_fitness_calls = fitness_instance.number_of_calls
            best_individual_of_generation = min(GA_population) if not maximization else max(GA_population)
            best_logistic_fitness = fitness_instance._calculate_logistic_fitness(best_individual_of_generation.genome)
            best_social_fitness = fitness_instance._calculate_social_fitness(best_individual_of_generation.genome)
            row = [n_fitness_calls, float(best_individual_of_generation), best_logistic_fitness, best_social_fitness]
            logs.append(row)
        
            logs_df = pd.DataFrame(logs, columns=['n_fitness_calls', 'best_individual_fitness', 'best_logistic_fitness', 'best_social_fitness'])
            logs_df['cumulative_calls'] = logs_df['n_fitness_calls'].cumsum()
            
            # add here the savin of the logs
            #logs_df.to_csv('logs.csv', index=False)
            
       
    best_individual = min(GA_population) if not maximization else max(GA_population)
    if save_logs:
        return best_individual, logs_df
    return best_individual

# Example usage
best_solution = genetic_algorithm(
    gen_count=100,
    selection_algorithm=tournament_selection,
    mutation_algorithms=[logistic_mutation, social_mutation],
    crossover_algorithms=[logistic_crossover_2, social_crossover],
    fitness_instance=my_fitness,
    Genome_class=Genome,
    maximization=False,
    xo_prob=0.9,
    mut_prob=0.2,
    social_mutation_prob=0.2,
    social_crossover_prob=0.2,
    population_size=10,
    elitism=True
)
print("Best Solution Genome:", best_solution.genome)
print("Best Solution Fitness:", float(best_solution))
print("Average distance:", best_solution.fitness_instance._calculate_social_fitness(best_solution.genome))



Runnig genetic alogrithm: 100%|██████████| 100/100 [00:03<00:00, 25.71 generation/s]

Best Solution Genome: 112-1002345-16-1028197531-1-17042986904672-1513-18
Best Solution Fitness: 1.8004041364554857
Average distance: 1.6


In [4]:
import itertools


parameter_grids = {
    
    'gen_count': [100, 200],
    'selection_algorithm': [tournament_selection, rank_selection],
    'mutation_algorithms': [[logistic_mutation, social_mutation], [logistic_mutation_2, social_mutation]],
    'crossover_algorithms': [[logistic_crossover_2, social_crossover], [full_crossover,full_crossover]],
    'fitness_instance': [my_fitness],
    'Genome_class': [Genome],
    'maximization': [False],
    'xo_prob': [0.2, 0.3],
    'mut_prob': [0.2, 0.3],
    'social_mutation_prob': [0.2, 0.3],
    'social_crossover_prob': [0.2, 0.3],
    'population_size': [7, 10, 15],
    'elitism': [True, False],
    'save_logs': [True],
}

# Create a list to store the results
results = []
# Iterate over all combinations of parameters

def run_grid_search(parameter_grids):
    keys = list(parameter_grids.keys())
    values = [parameter_grids[key] for key in keys]
    combinations = [dict(zip(keys, v)) for v in itertools.product(*values)]
    
    for params in tqdm.tqdm(combinations, desc="Running grid search", unit=" combination"):
        best_solution, logs_df = genetic_algorithm(**params)
        results.append((params, best_solution, logs_df))
    
    return results    

results = run_grid_search(parameter_grids)

Running grid search: 100%|██████████| 1536/1536 [1:03:43<00:00,  2.49s/ combination]
